In [1]:
import os
import cv2
from tqdm import tqdm
import random
import numpy as np
import json
import matplotlib.pyplot as plt

In [2]:
os.chdir('/home/gptrapletti/ds/satellite-multiclass-segm')

In [3]:
image_filepaths = sorted([os.path.join('data/images', filename) for filename in os.listdir(os.path.join('data/images'))])
mask_filepaths = sorted([os.path.join('data/masks', filename) for filename in os.listdir(os.path.join('data/masks'))])

len(image_filepaths), len(mask_filepaths)

(400, 400)

In [17]:
# Check image sizes
sizes = []
for i in tqdm(range(len(image_filepaths))):
    image = cv2.imread(image_filepaths[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_filepaths[i])
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    size = (image.shape, mask.shape)
    sizes.append(size)
    
set(sizes)

100%|██████████| 400/400 [04:13<00:00,  1.58it/s]


{((4000, 6000, 3), (4000, 6000, 3))}

All the images have the same size.

In [4]:
category_colors = {
    "unlabeled": (0, 0, 0),
    "paved-area": (128, 64, 128),
    "dirt": (130, 76, 0),
    "grass": (0, 102, 0),
    "gravel": (112, 103, 87),
    "water": (28, 42, 168),
    "rocks": (48, 41, 30),
    "pool": (0, 50, 89),
    "vegetation": (107, 142, 35),
    "roof": (70, 70, 70),
    "wall": (102, 102, 156),
    "window": (254, 228, 12),
    "door": (254, 148, 12),
    "fence": (190, 153, 153),
    "fence-pole": (153, 153, 153),
    "person": (255, 22, 96),
    "dog": (102, 51, 0),
    "car": (9, 143, 150),
    "bicycle": (119, 11, 32),
    "tree": (51, 51, 0),
    "bald-tree": (190, 250, 190),
    "ar-marker": (112, 150, 146),
    "obstacle": (2, 135, 115),
    "conflicting": (255, 0, 0)
}

For each class, look for 3 random images showing that class, turn everything to black and then plot.

In [ ]:
random_idxs = list(range(len(mask_filepaths)))

category_examples = {}
for category in tqdm(category_colors):
    random.shuffle(random_idxs)
    example_idxs = []
    n = 0
    for i in random_idxs:
        n += 1
        mask = cv2.imread(mask_filepaths[i])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        has_color = np.any(np.all(mask == category_colors[category], axis=2)) # if mask == [128, 64, 128] across color channels
        # If it has at least a pixel with the object class color
        if has_color:
            example_idxs.append(i)
        if len(example_idxs) == 3: # examples found
            category_examples[category] = example_idxs           
            break
        elif i == random_idxs[-1]: # no examples found
            break

In [9]:
# # Saver dict
# with open('data/category_examples.json', 'w') as file:
#     json.dump(category_examples, file)

# Load dict
with open('data/category_examples.json', 'r') as file:
    category_example_idxs = json.load(file)

In [15]:
absent_categories = set(category_colors) - set(category_example_idxs)
print(f'Categories absent in the images are: {list(absent_categories)}')

Categories absent in the images are: ['conflicting']


In [83]:
# For each category, find 3 images with objects of that category, turn everything to background and save plots.
categories = sorted(list(category_example_idxs))

for category in tqdm(categories):
    fig, axs = plt.subplots(1, 3, figsize=(20, 6))
    for i, idx in enumerate(category_example_idxs[category]):
        # Load image
        image = cv2.imread(image_filepaths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (400, 300))
        # Load mask
        mask = cv2.imread(mask_filepaths[idx])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.resize(mask, (400, 300), interpolation=cv2.INTER_NEAREST)
        # Keep only category of interest in the mask
        filter = np.all(mask == category_colors[category], axis=2)
        filter = np.stack([filter]*3, axis=-1)
        # Filter image
        image_filtered = image * filter
        # Plot        
        axs[i].imshow(image_filtered)
        axs[i].axis('off')
        axs[i].set_title(category)
    
    fig.tight_layout()
    fig.savefig(f'plots/{category}.png')
    plt.close(fig)   

100%|██████████| 23/23 [00:48<00:00,  2.13s/it]


Categories grouping:
- ar-marker: background
- bald-tree: vegetation
- bicycle: vehicles
- car: vehicles
- dirt: X
- dog: background
- ...



In [ ]:
# PRIMA VERIFICA IL NUMERO TOTALE DI PIXEL PER OGNI CLASSE, PER INDIVIDUARE QUELLE PIU' RARE
# - init dict with keys the colors/category str and value count = 0
# - Load mask
# - For each color/category, count n pixels
# - add to main dict

In [5]:
dict_i = {key: 0 for key in category_colors}

In [92]:
for category in dict_i:
    count = np.sum(np.all(mask == category_colors[category], axis=2))
    dict_i[category] = count

dict_i

{'unlabeled': 536,
 'paved-area': 27274,
 'dirt': 69,
 'grass': 0,
 'gravel': 8893,
 'water': 20817,
 'rocks': 6331,
 'pool': 0,
 'vegetation': 8249,
 'roof': 42978,
 'wall': 753,
 'window': 802,
 'door': 0,
 'fence': 922,
 'fence-pole': 46,
 'person': 688,
 'dog': 0,
 'car': 0,
 'bicycle': 0,
 'tree': 0,
 'bald-tree': 375,
 'ar-marker': 841,
 'obstacle': 426,
 'conflicting': 0}

In [10]:
pixel_counts = {key: 0 for key in category_colors}

for i in tqdm(range(len(mask_filepaths))):
    # Load mask
    mask = cv2.imread(mask_filepaths[i])
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    mask = cv2.resize(mask, (400, 300), interpolation=cv2.INTER_NEAREST)
    # Init empty count dict for the mask
    mask_pixel_counts = {key: 0 for key in category_colors}
    # For each category count n pixels and add to mask dict
    for category in mask_pixel_counts:
        count = np.sum(np.all(mask == category_colors[category], axis=2))
        mask_pixel_counts[category] = count.item() # as int, not np.array
    # Add mask dict values to main dict
    for key in mask_pixel_counts:
        pixel_counts[key] += mask_pixel_counts[key]

100%|██████████| 400/400 [02:44<00:00,  2.43it/s]


In [19]:
with open('data/category_pixel_counts.json', 'w') as file:
    json.dump(pixel_counts, file)

In [ ]:
# {'unlabeled': 208265,
#  'paved-area': 18077031,
#  'dirt': 1532646,
#  'grass': 9572780,
#  'gravel': 3498789,
#  'water': 1060368,
#  'rocks': 344291,
#  'pool': 305976,
#  'vegetation': 3402091,
#  'roof': 3526005,
#  'wall': 1287064,
#  'window': 267951,
#  'door': 14928,
#  'fence': 458427,
#  'fence-pole': 25384,
#  'person': 504175,
#  'dog': 6824,
#  'car': 376627,
#  'bicycle': 103413,
#  'tree': 983037,
#  'bald-tree': 637444,
#  'ar-marker': 109145,
#  'obstacle': 1697339,
#  'conflicting': 0}